In [12]:
%reset -f
%matplotlib inline

import time

import cv2
import numpy as np
import matplotlib.pyplot as plt


SOURCE = 0
KEY_ESC = 27
KEY_Q = 113
EXIT_KEY = KEY_Q
N_POINTS = 100

In [13]:
class DenseOpticalFlow:
    def __init__(self):
        pass

In [15]:
# Capture webcam
cap = cv2.VideoCapture(SOURCE)

# Set video buffer size to 1 to prevent buffer filling up
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# First frame
_, frame = cap.read()
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Store optical flow for all images
accumulated_mean_flow = np.array([[.0, .0]])

while True:
    # Save previous frame
    prev_gray_frame = gray_frame
    
    # Get current frame
    ret, frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow between previous and current frame
    flow = cv2.calcOpticalFlowFarneback(
        prev_gray_frame, # old frame
        gray_frame,      # new frame
        flow=None,       # where to save the flow matrix, not needed since we are returning
        pyr_scale=0.5,   # image scale for the pyramid
        levels=3,        # number of pyramid layers
        winsize=15,      # size of window used by flow algorithm (higher <=> more blur)
        iterations=4,    # max iterations for each pyramid layer
        poly_n=5,        # using recommended value (see docs)
        poly_sigma=1.1,  # using recommended value (see docs)
        flags=0          # operational flags (see docs)
    )
    
    # Calculate mean flow in x and y dimensions 
    mean_flow_x = np.mean(flow[:, :, 0])
    mean_flow_y = np.mean(flow[:, :, 1])
    accumulated_mean_flow -= [mean_flow_x, mean_flow_y]
    
    # Shift image to compensate flow difference
    # https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_geometric_transformations/py_geometric_transformations.html#translation
    tx = accumulated_mean_flow[0][0]
    ty = accumulated_mean_flow[0][1]
    M = np.array([[1, 0, tx], [0, 1, ty]], dtype=np.float32)
    gray_frame_shifted = cv2.warpAffine(gray_frame, M, (gray_frame.shape[1], gray_frame.shape[0]))

    # Mirror image because OpenCV is flipped by default
    flipped = cv2.flip(gray_frame_shifted, 1)
    
    # Display shifted frame
    cv2.imshow("Video", gray_frame_shifted)

    # Close window if EXIT_KEY was pressed
    key = cv2.waitKey(30)
    if key == EXIT_KEY:
        break

cap.release()
cv2.destroyAllWindows()